# Pip installation

use pip to install the openCV software

In [1]:
#pip install opencv-python

use pip to install the NumPy software

In [2]:
#pip install numpy

# Task 1

Import necessary libraries

In [1]:
#import libraries
import cv2 
import numpy as np

The class function accepts the path of the video and detect the moving object detection using frame differencing with OpenCV

In [13]:
# the function accept the path of the library and get the 
class Car_detector():
    def __init__(self,filePath):
        self.filepath = filePath
    
    def get_background(self):
        #open video from a given path
        video = cv2.VideoCapture(self.filepath)
        # calculate the median with the 50 randaomly selected frames
        frame_indices = video.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
        # store the frames in array to store the frames
        frames = []
        
        for i in frame_indices:
            # set the frame id to read that particular frame
            video.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = video.read()
            frames.append(frame)
            
        # calculate the median with numpy median function
        median_frame = np.median(frames, axis=0).astype(np.uint8)
        return median_frame
    
    def frame_differencing(self):
        # convert background to grayscale and let it be the base frame
        background = cv2.cvtColor(self.get_background(),cv2.COLOR_BGR2GRAY)
        cap = cv2.VideoCapture(self.filepath)
        #height of the video frame to help in identifying the area of main street
        height = int(cap.get(4))

        while (cap.isOpened()):
            ret,frame = cap.read()

            if ret is True:
                # convert frame to grayscale
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                # find the difference between current frame and base frame
                frame_diff = cv2.absdiff(gray, background)
                #thresholding to convert the frame to binary
                ret,threshold_frame=cv2.threshold(frame_diff,50,255, cv2.THRESH_BINARY)
                #dilate the frame to get some more white area
                dilate_frame = cv2.dilate(threshold_frame,None,iterations=5)

                frameCopy = frame.copy()
                
                #find the contours around the white segmented areas 
                contours, _ = cv2.findContours(dilate_frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                for cnt in contours:
                    # remove noise by continuing only is the contour area us less than 500
                    if cv2.contourArea(cnt) < 500:
                        continue
                    # get the xmin, ymin, width, and height coordinates from the contours
                    (x,y,w,h) = cv2.boundingRect(cnt)
                    # draw the bouding boxes only if the car is found on the main street
                    # and if the area is bigger than 6000
                    if y > height/2 and w*h > 6000:
                        cv2.rectangle(frameCopy,(x,y),(x +w,y+h),(0,255,0),2)


                #cv2.imshow('MASK frame', erode)
                cv2.imshow('FG MASK frame', frameCopy)


                # Wait until a key is pressed.
                # Retreive the ASCII code of the key pressed
                k = cv2.waitKey(1) & 0xff

                # Check if 'q' key is pressed.
                if k == ord('q'):

                    # Break the loop.
                    break
            else:
                break

        # Release the VideoCapture Object.
        cap.release()

        # Close the windows.q
        cv2.destroyAllWindows() 

In the Car_detector class function, two functions are defined which is get_background() and frame_differencing(). For the get_background function, the function is used to get the background model of a video where it will randomly select a few frames from the video file and calculate the median of those frames. The background model will be returned and it can be use for frame differencing. With the 50 randomly selected frame, the index position is set to the indices and the frames will be read. Those frames will be proceed to be stored in the frames list. The median frame will then be calculated from the frames list and return the background model found.

For the frame_differencing function, it will be used to detect the moving object by comparing the frame difference between the background model and current frame. Video file is read with the cv2.VideoCapture function.  While the reading of video information is true, the frames in the video is loop over where we find the difference between the current frame and the background model. Both the background model and currrent frame are converted to grayscale color format as frame differencing between frames can only be done with the same colour state. Thresholding helps to convert the resulting frame to binary format. If the pixel value is less than 50, it will be set it 0 else it will be set to 255. Then, dilating the thresholding result where we expands the white pixel regions. 5 iterations is used so that the white patches of a car will be more obvious and it will reduce the probabilty of having multiple rectangles around the same moving cars. This helps in removing the noise from the resulting image. The contours of the moving object were then found, then go over the contours and if the contour area is less than 500 do not do anything. If not, extract the minimum x and y coordinates and the width and height for each contour. Then, draw rectangle on the area of the current contour if it is more than 6000 and if it is found on below half of the frame. This can help to filter out the noise such as passerby are not detected. 

In [14]:
detector1 = Car_detector('Traffic_Laramie_1.mp4')
detector1.frame_differencing()

In [15]:
detector2 = Car_detector('Traffic_Laramie_2.mp4')
detector2.frame_differencing()

# Task 2

Import necessary libraries

In [18]:
#import library
from datetime import datetime
import cv2 
import pandas as pd

The class function MovingBbox() is used to keep the information about the movements of the bounding boxes. The city_car_counter() function will be used to detect the moving cars with background subtraction and count the number of cars that go from the city's downtown to the city centre.

In [19]:
#keep bounding boxes movement infomartion
class MovingBbox:
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        #moving direction
        self.dir = ''
        #if the box is had been counted
        self.counted = False
        #number of frames the bounding box has been tracked along
        self.numOfFrames = 0
        
def city_car_counter(filepath):
    #open video from given path
    video = cv2.VideoCapture(filepath)
    #calculate the precise number of cars per seconds with fps
    fps = video.get(cv2.CAP_PROP_FPS)
    #initialize the knn background object
    backgroundObject = cv2.createBackgroundSubtractorKNN(detectShadows = False)
    # store bounding boxes from previous frame
    prev_bbox = []
    #number of cars per min going towards city centre
    carsPerMin = 0
    start_time = datetime.now()
    # counter for number of cars
    counter = 0
    
    while (video.isOpened()):
        #read a new frame
        ret,frame = video.read()
        
        # Check if frame is not read correctly.
        if not ret:
            # Break the loop.
            break
        # Apply the background object on the frame to get the segmented mask. 
        fgmask = backgroundObject.apply(frame)
        # Apply some morphological operations to make sure you have a good mask
        fgmask = cv2.dilate(fgmask, None, 18)
        fgmask = cv2.erode(fgmask, None, 10)
        
        ret,threshold_frame=cv2.threshold(fgmask,100,255, cv2.THRESH_BINARY)
        
        #Detect contours in the frame.
        contours,_=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        #identify the bounding boxes in current frame
        curr_bbox = []
        
        for cnt in contours:
            (x,y,w,h) = cv2.boundingRect(cnt)
            #filter out human shapes and any small contours
            if (h/w >1.5) | (cv2.contourArea(cnt) < 500):
                continue
            if w*h > 6000:
                # add the bounding boxes for cars to current frame bbox
                curr_bbox.append(MovingBbox(x, y, w, h))
        
        #check if bboxes is found in prevBboxes
        for cbox in curr_bbox:
            exists = False
            
            for pbox in prev_bbox:
                #distance between bounding boxes from previous to current frame
                deltaX = cbox.x - pbox.x
                deltaY = cbox.y - pbox.y

                #difference in size from previous to current frame
                deltaW = cbox.w - pbox.w
                deltaH = cbox.h - pbox.h
                
                #determine whether the current bbox is too far from previous bbox
                if (abs(deltaX) > 10) | (abs(deltaY) > 10) | (abs(deltaW) > 10) |  (abs(deltaH) > 10):
                    continue
                
                exists = True
                #find the direction on X
                if deltaX > 0:
                    cbox.dir = 'right'
                elif deltaX < 0:
                    cbox.dir = 'left'
                    cbox.numOfFrames = pbox.numOfFrames + 1
                else:
                    cbox.dir = ''
                    
                #keep track of counted bboxed from frame to frame
                cbox.counted = pbox.counted
                
                #if the car come from the right frame and exit from the left frame
                if (cbox.x < 200) and (cbox.numOfFrames > 10) and (cbox.dir == 'left') and (cbox.counted == False):
                    counter+=1
                    cbox.counted = True
            
            # if previous bbox found, skip the remaining
            if exists ==True:
                continue
                
        #delta time from the beginning of the video up to the current frame
        deltaTime = datetime.now() - start_time 
        #number of seconds from the beginning
        secs = deltaTime.total_seconds()
        #number of cars detected per minute
        carsPerMin = counter * 60 / secs
        
        
        #Draw information on screen
        cv2.putText(frame, f'Count:{counter} in {int(secs)}s - ({carsPerMin:.2f} cars/min)', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2, cv2.LINE_AA)
        
    
        #assign current Bboxes to previous
        prev_bbox = curr_bbox

        
        #Show frames on screen
        cv2.imshow('Clean Mask', frame)


        # Wait until a key is pressed.
        # Retreive the ASCII code of the key pressed
        k = cv2.waitKey(1) & 0xff

        # Check if 'q' key is pressed.
        if k == ord('q'):

            # Break the loop.
            break
    

    # After the loop release the video object
    video.release()

    # Destroy all the windows
    cv2.destroyAllWindows()
    
    return counter,carsPerMin
    

The background subtraction used is KNN where shadows will be detected which means it will be shown as grey pixel. Firstly, an object is created to signify the KNN algorithm we are using for background subtraction. After every frame is read, apply() function is used in every frame of the video to remove the background. Dilate and erode operation is used to isolate the individual elements and joining disparate elements in the frame after removing the foreground. Dilation will be performed 18 times while erosion will be performed 10 times. Thresholding operation will convert the video frame to a binary image where is a particular image is greater than 100 it will be assigned the value to 255 otherwise 0. FindContours function is used to find the contours in the video frame. An empty array to store the current boxes is defined, then for each contour we will use the boundingRect to extract the minimum x and y coordinates and the width and height for each contour. With this we then filter out the human shapes and the small contours found. The moving object which is the car found will be appended to the array that store the boxes identified in the current frame. The MovingBox class function is used here before appending it, so that the information of the moving object can be tracked.

For each of the bounding boxes in that array it will check across the empty array that store the boxes in previous frame, if the boxes is found calculate the movement direction and store it if not it is a new bounding box then do not do anything. for the bounding boxes in prev array, we will find the difference in size and the distance between the two boxes so that we can make sure it is the same boxes. if the x-axis or the bounding boxes is decreasing means it is moving left else it is moving right. we will count the car if it exit the frame when the direction of the moving box is left, if the boxes have not been counted, the number of frame that the bounding box has been tracked along is more than 10 and the x axis of the moving box is lesser than 200. for the car per minute, it is used by the length of the video over the how many cars is found.


In [20]:
v1_count,v1_carPerMin = city_car_counter('Traffic_Laramie_1.mp4')

In [21]:
v2_count,v2_carPerMin = city_car_counter('Traffic_Laramie_2.mp4')

In [23]:
data = [["Traffic_Laramie_1.mp4", v1_count, round(v1_carPerMin,2)], ["Traffic_Laramie_2.mp4", v2_count,round(v2_carPerMin,2)]]
pd.DataFrame(data, columns=["", "Total Number of Cars", "Cars per Minute"])

,,Total Number of Cars,Cars per Minute
0,Traffic_Laramie_1.mp4,6,3.47
1,Traffic_Laramie_2.mp4,4,3.85
